In [17]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import applications
from keras import optimizers
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import backend as K
from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import glob
import matplotlib.pyplot as plt
from skimage.io import imread
from keras.callbacks import ModelCheckpoint
from skimage import exposure
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

In [24]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '../data/model/key19/train/pos6'
validation_data_dir = '../data/model/key19/validation/pos6'
nb_train_samples = int(len(glob.glob(train_data_dir+'/*')))
nb_validation_samples = int(len(glob.glob(validation_data_dir+'/*')))
epochs = 100
batch_size = 16
print('Found {} training samples, {} validation samples'.format(nb_train_samples,nb_validation_samples))

train2_data_dir = '../data/model/key19/train'
validation2_data_dir = '../data/model/key19/validation'
nb_train2_samples = int(len(glob.glob(train2_data_dir+'/*/*')))
nb_validation2_samples = int(len(glob.glob(validation2_data_dir+'/*/*')))
print('Found {} false training samples, {} false validation samples'.format(nb_train2_samples-nb_train_samples,nb_validation2_samples-nb_validation_samples))

Found 99 training samples, 47 validation samples
Found 330 false training samples, 95 false validation samples


In [25]:
train_path = glob.glob(train2_data_dir+'/*/*')
val_path   = glob.glob(validation2_data_dir+'/*/*')

In [26]:
x_train = []
for image in train_path:
    img = load_img(image,target_size=(img_width,img_height))  # load image
    x1 = img_to_array(img)  # convert to numpy shape (3, 150, 150)
    #x = x.reshape((1,) + x.shape)  # rearrange to shape (1, 3, 150, 150)
    x_train.append(x1)
x_train = np.array(x_train)

x_val = []
for image in val_path:
    img = load_img(image,target_size=(img_width,img_height))  # load image
    x1 = img_to_array(img)  # convert to numpy shape (3, 150, 150)
    #x = x.reshape((1,) + x.shape)  # rearrange to shape (1, 3, 150, 150)
    x_val.append(x1)
x_val = np.array(x_val)

In [27]:
y_train = []
for i in range(0,nb_train2_samples):
    if i < nb_train_samples:
        y_train.append(1)
    else:
        y_train.append(0)
y_train = np.array(y_train)
y_val = []
for i in range(0,nb_validation2_samples):
    if i < nb_validation_samples:
        y_val.append(1)
    else:
        y_val.append(0)
y_val = np.array(y_val)

In [28]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255.
x_val /= 255.

In [29]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [37]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(1, activation='sigmoid'))

add_model.summary()

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 6,423,041
Trainable params: 6,423,041
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________


In [31]:
train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)

In [46]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val))

Epoch 1/100
26/26 [==============================] - 13s - loss: 0.5220 - acc: 0.7500 - val_loss: 1.0538 - val_acc: 0.6549
Epoch 2/100
 3/26 [==>...........................] - ETA: 10s - loss: 0.2824 - acc: 0.7869

KeyboardInterrupt: 

In [47]:
from vis.losses import ActivationMaximization
from vis.regularizers import TotalVariation, LPNorm
from vis.input_modifiers import Jitter
from vis.optimizer import Optimizer

from vis.callbacks import GifGenerator


# The name of the layer we want to visualize
# (see model definition in vggnet.py)
layer_name = 'sequential_2'
#layer
#layer_dict = dict([(layer.'sequential_2'.'dense_6', layer) for layer in model.layers[1:]])
output_class = [1]

losses = [
    (ActivationMaximization(model.layers[-1].layers[-1], output_class), 2),
    (LPNorm(model.input), 10),
    (TotalVariation(model.input), 10)
]
opt = Optimizer(model.input, losses)
opt.minimize(max_iter=500, verbose=True, image_modifiers=[Jitter()], callbacks=[GifGenerator('opt_progress')])

ValueError: slice index 1 of dimension 1 out of bounds. for 'strided_slice_1' (op: 'StridedSlice') with input shapes: [?,1], [2], [2], [2] and with computed input tensors: input[1] = <0 1>, input[2] = <0 2>, input[3] = <1 1>.

In [43]:
print(model.layers[-1].layers[-1].name)

dense_6


In [ ]:
layer_dict[layer_name]